# Compute statistically significant fluxes between groups - REDS Recall
## Setup
### Import packages

In [ ]:
import re
import textwrap
import warnings
from collections import defaultdict
from itertools import combinations

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rbc_gem_utils import (
    COBRA_CONFIGURATION,
    ensure_iterable,
    get_dirpath,
    read_cobra_model,
    show_versions,
)
from rbc_gem_utils.analysis.overlay import (
    DEFAULT_PREFIX_SUFFIX_VALUES,
    DEFAULT_PROTEOME_COMPARTMENT,
    EnzymeDilution,
    add_relaxation_budget,
    load_overlay_model,
    plot_correlations,
)
from rbc_gem_utils.visualization import cmap_map
from scipy.stats import false_discovery_control, kruskal, mannwhitneyu, spearmanr

plt.rcParams["font.family"] = "Arial"

show_versions()

### Define configuration
#### COBRA Configuration

In [ ]:
COBRA_CONFIGURATION.solver = "gurobi"
# Set bound defaults much larger to prevent model loading issues
COBRA_CONFIGURATION.bounds = (-1e-8, 1e8)
COBRA_CONFIGURATION.tolerance = 1e-9
COBRA_CONFIGURATION

### Define organism, model, and dataset

In [ ]:
organism = "Human"
model_id = "RBC_GEM"
dataset_name = "REDSRecall"
grouped_data_key = "Sample"

### Set variables for sample identification

In [ ]:
# For sample IDs
sample_key = "SAMPLE ID"
donor_key = "PUBLIC RECALL DONOR ID"
time_key = "DAY"
timepoints = ["D10", "D23", "D42"]
genotypes = ["G6PD_V68M", "ATP11C_V972M"]
donor_re = re.compile(rf"(?P<donor>S(?P<num>\d\d\d))")
time_re = re.compile(rf"(?P<time>{'|'.join(timepoints)})")
genotype_re = re.compile(rf"(?P<genotype>({'|'.join(genotypes)}))")

operations = "|".join([x.capitalize() for x in ["mean", "median"]])

operation_re = re.compile(r"(?P<op>" + operations + r")\_(?P<group>\w+)")
sample_id_re = re.compile(
    r"(?!" + operations + r")" + donor_re.pattern + r"\_" + time_re.pattern
)

### Set computation options

In [ ]:
ftype = "xml"  # In our experience, SBML/XML loads faster, but will take up to 4x more space uncompressed as compared to JSON
run_computations = True  # Keep off to use previously computed results
overwrite = False  # Whether to allow overwriting of previous simulation results
verbose = True

# Objective reactions
objective_reactions = ["NaKt"]
# Reactions that must have the capability to carry flux, sort for consistency
required_flux_reactions = []  # Add reactions to this list
required_flux_reactions = sorted(set(objective_reactions + required_flux_reactions))


enzyme_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.dilution"]
enzyme_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.metabolite"]
enzyme_met_suffix_total = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["suffix.total"]
comp_suffix = f"_{DEFAULT_PROTEOME_COMPARTMENT}"

#### Set prefixes/suffixes to expect

In [ ]:
enzyme_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.dilution"]
enzyme_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.metabolite"]
enzyme_met_suffix_total = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["suffix.total"]
comp_suffix = f"_{DEFAULT_PROTEOME_COMPARTMENT}"

### Set figure options

In [ ]:
save_figures = True
transparent = False
imagetype = "svg"

### Set paths

In [ ]:
# Set paths
processed_data_dirpath = get_dirpath(use_temp="processed") / organism / dataset_name
overlay_dirpath = get_dirpath("analysis") / "OVERLAY" / organism
model_dirpath = overlay_dirpath / model_id
results_dirpath = (
    get_dirpath(use_temp="processed")
    / model_id
    / "OVERLAY"
    / organism
    / dataset_name
    / grouped_data_key
)
pcfva_results_dirpath = (
    results_dirpath
    / "pcFVA"
    / "_".join(("REQ", *required_flux_reactions))
    / "_".join(("OBJ", *objective_reactions))
)

# Objective reaction does not matter since correlations are computed
# based on min and max fluxes and abundance, which are obtained when optimum is 0.
corr_results_dirpath = results_dirpath / "correlations"
# Ensure directory  exists
corr_results_dirpath.mkdir(exist_ok=True, parents=True)

## Load RBC-GEM model

In [ ]:
model = read_cobra_model(filename=model_dirpath / f"{model_id}.xml")
pcmodel = load_overlay_model(filename=model_dirpath / f"{model_id}_PC.xml")

# Add relaxation budget to initial PC model to get names of relaxation reactions
add_relaxation_budget(pcmodel, 0, verbose=False)
pcmodel

## Load pcFVA generated results

In [ ]:
# Load DataFrame of generated results
df_pcfva_all = pd.read_csv(
    pcfva_results_dirpath / f"{pcmodel.id}_All_FVAsols.zip",
    index_col=None,
).fillna(0)

df_pcfva_all

## Create DataFrame for calculations and visualizations
### Get maximum reaction fluxes and associated abundance values
#### Get maximum reaction fluxes and ranges

In [ ]:
rxns = model.reactions.query(lambda x: len(x.genes)).list_attr("id")
df_max_flux_per_model = df_pcfva_all[df_pcfva_all["reactions"].isin(rxns)].copy()
df_max_flux_per_model = df_max_flux_per_model.groupby(
    ["model", "reactions", "optimum"]
)[["min", "max"]].agg(
    {
        "min": "min",  # Minimum reaction flux per model
        "max": "max",  # Maximum reaction flux per model
    }
)
# Address issues possibly caused by floating point precision, ideally a value that prevents any negative ranges
df_max_flux_per_model.loc[
    df_max_flux_per_model["max"] < df_max_flux_per_model["min"], ["max", "min"]
] = [0, 0]
atol = COBRA_CONFIGURATION.tolerance * 100
df_max_flux_per_model["max"] = df_max_flux_per_model["max"].apply(
    lambda x: 0 if np.isclose(x, 0, atol=atol) else round(x, -int(np.log10(atol)))
)
df_max_flux_per_model["min"] = df_max_flux_per_model["min"].apply(
    lambda x: 0 if np.isclose(x, 0, atol=atol) else round(x, -int(np.log10(atol)))
)
df_max_flux_per_model["range"] = (
    df_max_flux_per_model["max"] - df_max_flux_per_model["min"]
)
# Ensure no negative values, if results appear then tolerance should be adjusted
df_max_flux_per_model[df_max_flux_per_model["range"] < 0]

#### Get maximum "enzyme" abundances

In [ ]:
rxns = pcmodel.reactions.query(
    lambda x: isinstance(x, EnzymeDilution)
    and x.id.endswith(f"{enzyme_met_suffix_total}{comp_suffix}")
).list_attr("id")
df_max_abundance_per_model = df_pcfva_all[df_pcfva_all["reactions"].isin(rxns)].copy()
# Rename dilution reactions to match
reaction_enzyme_map = {
    enzyme_rid: enzyme_rid.replace(
        f"{enzyme_rxn_prefix}{enzyme_met_prefix}", ""
    ).replace(f"{enzyme_met_suffix_total}{comp_suffix}", "")
    for enzyme_rid in df_max_abundance_per_model["reactions"]
}
df_max_abundance_per_model["reactions"] = df_max_abundance_per_model[
    "reactions"
].replace(reaction_enzyme_map)
df_max_abundance_per_model = df_max_abundance_per_model.groupby(
    ["model", "reactions", "optimum"]
)[["max"]].max()
# Address issues possibly caused by floating point precision, atol is ideally a value that prevents any negative ranges
atol = COBRA_CONFIGURATION.tolerance
df_max_abundance_per_model["max"] = df_max_abundance_per_model["max"].apply(
    lambda x: 0 if x < 0 else x
)
df_max_abundance_per_model["max"] = df_max_abundance_per_model["max"].apply(
    lambda x: 0 if np.isclose(x, 0, atol=atol) else round(x, -int(np.log10(atol)))
)
df_max_abundance_per_model = df_max_abundance_per_model.rename(
    {"max": "abundance"}, axis=1
)
# Ensure no negative values, if results appear then tolerance should be adjusted
df_max_abundance_per_model[(df_max_abundance_per_model < 0).any(axis=1)]

#### Merge DataFrames

In [ ]:
df_data_all = pd.merge(
    df_max_flux_per_model,
    df_max_abundance_per_model,
    left_index=True,
    right_index=True,
    how="left",
)
df_data_all = df_data_all.reset_index(drop=False)
df_data_all

### Identify donor, timepoints, and genotypes for results

In [ ]:
df_metadata = pd.read_csv(
    processed_data_dirpath / "Metadata.csv",
    index_col=[sample_key],
).convert_dtypes()
df_metadata = df_metadata.loc[:, genotypes]

df = df_metadata.reset_index(drop=False)
df[sample_key] = df[sample_key].str.split("_", expand=True).iloc[:, 0]
df = df.drop_duplicates().set_index(sample_key)
for col, series in df.items():
    print(series.value_counts().sort_index())
    print()

In [ ]:
metadata_columns = ["sample", "donor", "time", "genotype"]
for key, search_re in zip(
    metadata_columns, [sample_id_re, donor_re, time_re, genotype_re]
):
    if key == "sample":
        df_data_all[key] = df_data_all["model"].apply(
            lambda x: x.replace(f"{pcmodel.id}_", "")
        )
    else:
        df_data_all[key] = df_data_all["model"].apply(
            lambda x: search_re.search(x).group(1) if search_re.search(x) else pd.NA
        )

df_data_all = df_data_all.merge(
    df_metadata, left_on="sample", right_index=True, how="left"
)

# Add genotypes alleles
for genotype in genotypes:
    df = df_data_all[df_data_all["genotype"] == genotype]["sample"]
    df = df.str.rsplit("_", n=1, expand=True)
    if df.empty:
        continue
    df = df.iloc[:, -1]
    df_data_all.loc[df.index, genotype] = df.values
df_data_all = df_data_all.drop("genotype", axis=1)
metadata_columns = metadata_columns[:-1] + genotypes
df_data_all

## Compute statistically significant results between groups
### Remove models based on data operations

In [ ]:
df_data_samples = df_data_all[
    [not bool(operation_re.search(x)) for x in df_data_all["model"]]
].reset_index(drop=True)
df_data_samples

### Combine sample results based on donor genotype

In [ ]:
id_key = "sample"
groupby_operation = None

In [ ]:
if groupby_operation is None:
    df_data_for_analyses = df_data_samples.copy()
elif groupby_operation == "mean":
    df_data_for_analyses = df_data_samples.groupby(
        ["reactions", id_key, "optimum"], as_index=False
    )[["min", "max", "range", "abundance"] + genotypes].mean()
elif groupby_operation == "median":
    df_data_for_analyses = df_data_samples.groupby(
        ["reactions", id_key, "optimum"], as_index=False
    )[["min", "max", "range", "abundance"] + genotypes].median()
else:
    raise ValueError(f"Unrecognized operation to perform: '{groupby_operation}'")
df = df_data_for_analyses[genotypes].dropna().astype(int)
df_data_for_analyses.loc[df.index, genotypes] = df
df_metadata = (
    df_data_for_analyses[[id_key] + genotypes].drop_duplicates().set_index(id_key)
)
df_data_for_analyses

### Create groups of models

In [ ]:
all_key = "ALL"
model_groups = {all_key: list(df_data_for_analyses[id_key].unique())}


def create_group_of_models(df, id_key, groupby, verbose=False):
    grouped = df.groupby(groupby)[id_key].agg(lambda x: list(x.unique()))
    grouped = {
        "_".join([str(x) for x in ensure_iterable(k)]): v
        for k, v in grouped.to_dict().items()
    }
    if verbose:
        max_name_len = max([len(group_name) for group_name in list(grouped)])
        for group_name, model_list in grouped.items():
            spacepad = "".join([" "] * (max_name_len - len(group_name)))
            print(f"{group_name}:{spacepad}\t{len(model_list)} samples")
    return grouped

#### Based on genotype

In [ ]:
for genotype in genotypes:
    grouped = create_group_of_models(
        df_data_for_analyses, id_key=id_key, groupby=genotype, verbose=False
    )
    grouped = {f"{genotype}_{str(k)}": v for k, v in grouped.items()}
    if verbose:
        max_name_len = max([len(group_name) for group_name in list(grouped)])
        for group_name, model_list in grouped.items():
            spacepad = "".join([" "] * (max_name_len - len(group_name)))
            print(f"{group_name}:{spacepad}\t{len(model_list)} samples")
    model_groups.update(grouped)
    print()

### View groups

In [ ]:
print("Possible groups for analyses\n============================")
max_name_len = max([len(group_name) for group_name in list(model_groups)])
for group_name, model_list in model_groups.items():
    spacepad = "".join([" "] * (max_name_len - len(group_name)))
    print(f"{group_name}:{spacepad}\t{len(model_list)} samples")

df_data_for_analyses = df_data_for_analyses.set_index(["reactions", id_key])
df_data_for_analyses

#### Ensure groups exist and setup directory structure

In [ ]:
groups_dict = defaultdict(dict)
item_list = genotypes
groups_dict[all_key].update({item: {} for item in item_list})

header = "Expected directory structure"
print("\n".join((header, "=" * len(header), all_key)))
for idx, (group_name, subgroups) in enumerate(sorted(groups_dict[all_key].items())):
    print("\u2514\u2500\u2500" + f" {group_name}")

group_results_dirpath_dict = {all_key: corr_results_dirpath}
for group_name, subgroups in groups_dict[all_key].items():
    group_results_dirpath_dict[group_name] = (
        group_results_dirpath_dict[all_key] / group_name
    )

#### Load subsystems and metabolic categories to enrich results

In [ ]:
subsystems_to_exclude = {"Pseudoreactions"}
use_abbrevs = True
abbreviations = {
    "Amino acid metabolism": "A",
    "Carbohydrate metabolism": "C",
    "Lipid metabolism": "L",
    "Metabolism of cofactors and vitamins": "V",
    "Nucleotide metabolism": "N",
    "Reactive species": "R",
    "Transport reactions": "T",
    "Other": "O",
}
categories_to_keep = list(abbreviations)

df_pathways = pd.read_csv(
    get_dirpath("curation") / "subsystems.tsv", sep="\t", dtype=str
).fillna("")

# Rename "name" to subsystem to match reaction attribute
df_pathways = df_pathways.rename({"name": "subsystem"}, axis=1)
# Group "Metabolism of other amino acids" with amino acids rather than treat as "other"
df_pathways["category"] = df_pathways["category"].replace(
    "Metabolism of other amino acids", "Amino acid metabolism"
)

df_pathways["category"] = df_pathways["category"].apply(
    lambda x: ("Other" if x not in categories_to_keep else x)
)
df_pathways = df_pathways[~df_pathways["subsystem"].isin(subsystems_to_exclude)].copy()
subsystem_to_category_dict = df_pathways.set_index("subsystem")["category"].to_dict()
df_pathways

## Compute significant results between groups
#### Compare all subgroups at once

In [ ]:
compare_group = "ATP11C_V972M"
optimum = 0
value_to_compare = "range"
group_timepoints_by = None
compare_pairwise = True
compare_all_groups = True
if compare_group in genotypes:
    ordered_group_to_compare = [f"{compare_group}_{alleles}" for alleles in [0, 1, 2]]
else:
    raise NotImplementedError
all_samples_for_comparison = [
    value for g in ordered_group_to_compare for value in np.array(model_groups[g])
]
df_data_for_correlations = df_data_for_analyses.loc[
    pd.IndexSlice[:, all_samples_for_comparison], :
]
df_data_for_correlations = df_data_for_correlations[
    df_data_for_correlations["optimum"] == optimum
].drop("optimum", axis=1)

print("Groups to compare\n=================")
pairwise_group_combos = []
if compare_all_groups:
    print(tuple(ordered_group_to_compare))
if compare_pairwise:
    pairwise_group_combos += list(combinations(ordered_group_to_compare, 2))
    for group in pairwise_group_combos:
        print(group)
df_data_for_correlations

### Kruskal Wallis H-test (3 or more groups)

In [ ]:
results_dict = defaultdict(dict)
sample_size_too_small = defaultdict(set)

In [ ]:
if len(ordered_group_to_compare) > 2:
    for rid in df_data_for_correlations.index.get_level_values("reactions").unique():
        df_data_rxn = df_data_for_correlations.loc[rid]
        df_data_rxn_opt_value = df_data_rxn[value_to_compare].copy()
        data_arrays = {
            group_name: df_data_rxn_opt_value.loc[model_groups[group_name]].values
            for group_name in ordered_group_to_compare
        }
        values = list(data_arrays.values())
        unique_values = set(
            [v for value_list in values for v in value_list if not np.isnan(v)]
        )
        if any([len(v[~np.isnan(v)]) < 6 for v in values]):
            sample_size_too_small[tuple(ordered_group_to_compare)].add(rid)
            print(rid)
            break
        if len(unique_values) <= 1:
            # Skip variables that do not have any differences
            results_dict[tuple(ordered_group_to_compare)][rid] = dict(
                zip(["statistic", "pvalue"], [pd.NA, pd.NA])
            )
        else:
            result = kruskal(*values, nan_policy="omit")
            results_dict[tuple(ordered_group_to_compare)][rid] = {
                attr: getattr(result, attr) for attr in ["statistic", "pvalue"]
            }
dataframes = {
    key: pd.DataFrame.from_dict(values, orient="index")
    for key, values in results_dict.items()
}

### Mann Whiteney U test (2 groups)

In [ ]:
if len(ordered_group_to_compare) == 2 or compare_pairwise:
    for rid in df_data_for_correlations.index.get_level_values("reactions").unique():
        df_data_rxn = df_data_for_correlations.loc[rid]
        df_data_rxn_opt_value = df_data_rxn[value_to_compare].copy()
        data_arrays = {
            group_name: df_data_rxn_opt_value.loc[model_groups[group_name]].values
            for group_name in ordered_group_to_compare
        }
        combos = (
            [tuple(ordered_group_to_compare)]
            if not pairwise_group_combos
            else pairwise_group_combos
        )
        for combo in combos:
            values = [data_arrays[group] for group in combo]
            unique_values = set(
                [v for value_list in values for v in value_list if not np.isnan(v)]
            )
            if any([len(v[~np.isnan(v)]) < 6 for v in values]):
                sample_size_too_small[tuple(combo)].add(rid)
                print(rid)
            if len(unique_values) <= 1:
                # Skip variables that do not have any differences
                result = dict(zip(["statistic", "pvalue"], [pd.NA, pd.NA]))
            else:
                result = mannwhitneyu(*values, nan_policy="omit", method="auto")
                result = {
                    attr: getattr(result, attr) for attr in ["statistic", "pvalue"]
                }
            results_dict[combo][rid] = result
dataframes = {
    key: pd.DataFrame.from_dict(values, orient="index")
    for key, values in results_dict.items()
}
print(f"Number of different comparisons made: {len(dataframes)}")
print("Groups compared\n===============")
for key in list(dataframes):
    print(key)

### Determine significance using p-values

In [ ]:
pvalue_sig = 0.05
enzyme_reactions_only = False
include_boundary_reactions = False
sort_by_subsystem = False
standardize_by = "mean"
use_group_means = False
fdr_method = None

significant_dataframes = {}
for met in model.metabolites.query(lambda x: x.compartment == "e"):
    met.name += " (extracellular)"
metadata_columns = [
    "name",
    "stoichiometry",
    "proteins",
    "pvalue" if not fdr_method else "adj_pvalue",
    "subsystem",
    "category",
]
for key, df in dataframes.items():
    df = df.dropna().copy()
    df["pvalue"] = df["pvalue"].astype(float)
    if fdr_method is not None and not fdr_method in {"bon", "bh", "by"}:
        raise ValueError(f"Unrecognized FDR correction method : {fdr_method}")
    elif fdr_method == "bon":
        pvalue_key = "adj_pvalue"
        df[pvalue_key] = df["pvalue"] * len(df["pvalue"])
    elif fdr_method in {"bh", "by"}:
        pvalue_key = "adj_pvalue"
        df[pvalue_key] = false_discovery_control(
            df["pvalue"].astype(float), method=fdr_method
        )
    else:
        pvalue_key = "pvalue"
    print(df.loc["PSFLIPt", "pvalue"])
    pvalue = pvalue_sig if isinstance(pvalue_sig, (float, int)) else pvalue_sig[key]
    df = df[df[pvalue_key] <= pvalue].drop("statistic", axis=1)
    if enzyme_reactions_only:
        df_pivot = df_data_for_correlations.loc[
            df.index, ["abundance", value_to_compare]
        ].dropna(subset="abundance")
        df_pivot = df_pivot.drop("abundance", axis=1)
    else:
        df_pivot = df_data_for_correlations.loc[df.index, value_to_compare]
    if not include_boundary_reactions:
        df_pivot = df_pivot[
            ~df_pivot.index.isin(
                model.reactions.query(lambda x: x.boundary).list_attr("id"),
                level="reactions",
            )
        ]
    df_pivot = df_pivot.reset_index(drop=False)
    df_pivot = df_pivot.pivot(
        columns=id_key, index="reactions", values=value_to_compare
    )
    df = pd.merge(df, df_pivot, left_index=True, right_index=True).sort_values(
        pvalue_key
    )
    df.index.name = "reactions"
    df = df.reset_index(drop=False).set_index(["reactions", pvalue_key]).T
    if df.empty:
        df = pd.DataFrame([], columns=metadata_columns)
    else:
        df = pd.concat(
            [
                # Sort index by donor number and subgroup while concatenating
                df.loc[model_groups[g]].sort_index()
                for g in key
            ],
            axis=0,
        )

        df = df.T.reset_index(drop=False)
        # Enrich results
        # df["name"] = [r.name.replace(",", "") for r in model.reactions.get_by_any(list(df["reactions"].values))]
        df["name"] = [
            r.name for r in model.reactions.get_by_any(list(df["reactions"].values))
        ]
        df["stoichiometry"] = [
            r.build_reaction_string(use_metabolite_names=True)
            for r in model.reactions.get_by_any(list(df["reactions"].values))
        ]
        df["subsystem"] = [
            r.subsystem
            for r in model.reactions.get_by_any(list(df["reactions"].values))
        ]
        df["category"] = df["subsystem"].replace(subsystem_to_category_dict)
        df["proteins"] = [
            ";".join(sorted([g.id for g in r.genes]))
            for r in model.reactions.get_by_any(list(df["reactions"].values))
        ]
        # Replace commas to prevent issues with CSV export
        # df["subsystem"] = df["subsystem"].apply(lambda x: x.replace(",", ""))
        # df["category"] = df["category"].apply(lambda x: x.replace(",", ""))
        df[pvalue_key] = df[pvalue_key].apply(lambda x: round(x, 5))

        df = df.set_index("reactions")
        if sort_by_subsystem:
            df = df.sort_values(by=["category", "subsystem", "proteins"])

        df_meta = df.loc[:, metadata_columns].copy()
        df_data = df.loc[:, ~df.columns.isin(df_meta.columns)].copy()
        if use_group_means:
            df_data = pd.concat(
                [df_data.loc[:, model_groups[g]].mean(axis=1) for g in key], axis=1
            )
            df_data.columns = list(key)
        if standardize_by == "mean":
            df_data = (
                df_data.sub(df_data.mean(axis=1), axis=0)
                .div(df_data.std(axis=1), axis=0)
                .dropna(how="all", axis=0)
            )
        elif standardize_by == "median":
            df_data = (
                (df_data.T - df_data.median(axis=1))
                / (df_data.quantile(q=0.75, axis=1) - df_data.quantile(q=0.25, axis=1))
            ).T
        else:
            df_data = df_data.loc[
                :, [x for x in df_metadata.index if x in df_data.columns]
            ]
        # Put dataframes back together for custom reordering
        df = df_data.merge(df_meta, left_index=True, right_index=True)
    significant_dataframes[key] = df
    print(key)
    print(f"Min & Max values: ({df_data.min().min():.4f}, {df_data.max().max():.4f})")
    print()
key = tuple(ordered_group_to_compare)
# key = ('ATP11C_V972M_0', 'ATP11C_V972M_2')
df = significant_dataframes[key]
df

## Export results

In [ ]:
ftype = "tsv"
for key, df_main in significant_dataframes.items():
    df_meta_row = df_main.loc[:, metadata_columns].copy()
    df_data = df_main.loc[:, ~df_main.columns.isin(df_meta.columns)].copy()
    df_meta_col = df_metadata.copy()
    if use_group_means:
        df_meta_col = df_meta_col.groupby(compare_group, as_index=False).mean()
        df_meta_col.index = [
            f"{genotype}_{x}" for x in df_meta_col[compare_group].values
        ]

    df_meta_col = df_meta_col.loc[list(df_data.columns)].copy()
    for df_type, df in zip(
        ["data", "meta_row", "meta_col"], [df_data, df_meta_row, df_meta_col]
    ):
        filename = "_".join(
            ["MannWhiteney" if len(key) == 2 else "Kruskal"]
            + [g.split("_")[-1] for g in key]
            + [id_key]
            + [df_type]
        )
        if use_group_means:
            filename += "_mean"
        group_results_dirpath_dict[group_name].mkdir(exist_ok=True)
        filename = group_results_dirpath_dict[group_name] / filename
        df.to_csv(
            f"{filename}.{ftype}", sep="\t" if ftype == "tsv" else ",", index=True
        )